This notebook can be greatly improved. There are too hard-coded values and too much duplicate code.

In [1]:
import sys

sys.path.append('..')

In [2]:
import sqlite3
import textwrap
from collections import namedtuple
from pathlib import Path
from pprint import pp

import pandas as pd
from IPython.display import display
from ipywidgets import interact
from PIL import Image, ImageColor, ImageDraw, ImageFont
from sklearn import metrics

from herbarium.pylib import db

In [3]:
DATA = Path('..') / 'data'

DB = DATA / 'angiosperms.sqlite'
IMAGES = DATA / 'images'

In [4]:
TEST_RUNS = """ b0_flowers_all_orders_1 b0_fruits_all_orders_1 """.split()

FIELDS = [
    ('reproductivecondition', 'reproductive condition'),
    ('occurrenceremarks', 'occurrence remarks'),
    ('fieldnotes', 'field notes'),
    ('dynamicproperties', 'dynamic properties'),
]

TRAITS = [
    ('flowering', 'not_flowering'),
    ('fruiting', 'not_fruiting'),
    # ('leaf_out', 'not_leaf_out'),
]

FLOWERING = db.select_test_run(DB, 'b0_flowers_all_orders_1')

FRUITING = db.select_test_run(DB, 'b0_fruits_all_orders_1')

In [8]:
pad = 8
color = 'black'

Text = namedtuple('Text', 'x y bbox text')

In [9]:
indent = ' ' * 24

def build_text(draw, font, texts, text, x, y):
    for t in textwrap.wrap(text, subsequent_indent=indent):
        bbox = draw.textbbox((0, 0), t, font, anchor='lt')
        texts.append(Text(x, y, bbox, t))
        y += bbox[3] + pad
    return y

# Flowering results

In [5]:
true_pos = [r for r in FLOWERING if r['true'] == 1 and r['true'] == round(r['pred'])]
true_neg = [r for r in FLOWERING if r['true'] == 0 and r['true'] == round(r['pred'])]
false_pos = [r for r in FLOWERING if r['true'] == 1 and r['true'] != round(r['pred'])]
false_neg = [r for r in FLOWERING if r['true'] == 0 and r['true'] != round(r['pred'])]

In [6]:
y_true = pd.Series([round(r['true']) for r in FLOWERING])
y_pred = pd.Series([round(r['pred']) for r in FLOWERING])

df_confusion = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'])
df_confusion

Predicted,0,1
Actual,,
0,284,19
1,110,2186


In [35]:
DATASET = true_neg
CONFUSION = 'true_neg'

DIR = DATA / 'temp' / CONFUSION
DIR.mkdir(exist_ok=True)

In [36]:
def flowering(idx):
    rec = DATASET[idx - 1]
    image = Image.open(Path('..') / rec['path'])

    font = ImageFont.truetype(str(DATA / 'fonts' / 'SourceCodePro-Regular.ttf'), 64)
    draw = ImageDraw.Draw(image)

    texts = []

    x, y = 10, 10
    w, h = image.size
    
    text = f'{"confusion:":<23} {CONFUSION}'
    y = build_text(draw, font, texts, text, x, y)

    text = f'{"coreid:":<23} {rec["coreid"]}'
    y = build_text(draw, font, texts, text, x, y)

    for field, label in FIELDS:
        text = f'{(label+":"):<23} {rec[field]}'
        y = build_text(draw, font, texts, text, x, y)

    flag = ''
    if rec['flowering']:
        flag += '1'
    elif rec['not_flowering']:
        flag += '0'

    text = f'{"flowering NLP:":<23} {flag}'
    y = build_text(draw, font, texts, text, x, y)

    text = f'{"flowering model:":<23} {round(rec["pred"])} ({rec["pred"]:0.4})'
    y = build_text(draw, font, texts, text, x, y)

    max_x = max(t.bbox[2] for t in texts)
    draw.rectangle((0, 0, max_x + 32, y + 32), fill='#eeeeee')

    for t in texts:
        draw.text((t.x, t.y), t.text, color, font=font)

    path = DIR / f'{rec["coreid"]}.jpg'
    image.save(path, 'JPEG')
    
    # display(image)


# flowering(19)
# interact(flowering, idx=(1, len(dataset)))

In [37]:
for i, _ in enumerate(DATASET[:20], 1):
    flowering(i)

# Fruiting results

In [38]:
true_pos = [r for r in FRUITING if r['true'] == 1 and r['true'] == round(r['pred'])]
true_neg = [r for r in FRUITING if r['true'] == 0 and r['true'] == round(r['pred'])]
false_pos = [r for r in FRUITING if r['true'] == 1 and r['true'] != round(r['pred'])]
false_neg = [r for r in FRUITING if r['true'] == 0 and r['true'] != round(r['pred'])]

In [39]:
y_true = pd.Series([round(r['true']) for r in FRUITING])
y_pred = pd.Series([round(r['pred']) for r in FRUITING])

df_confusion = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'])
df_confusion

Predicted,0,1
Actual,,
0,267,54
1,156,797


In [55]:
DATASET = false_neg
CONFUSION = 'false_neg'

DIR = DATA / 'temp' / CONFUSION
DIR.mkdir(exist_ok=True)

In [56]:
def fruiting(idx):
    rec = DATASET[idx - 1]
    image = Image.open(Path('..') / rec['path'])

    font = ImageFont.truetype(str(DATA / 'fonts' / 'SourceCodePro-Regular.ttf'), 64)
    draw = ImageDraw.Draw(image)

    texts = []

    x, y = 10, 10
    w, h = image.size
    
    text = f'{"confusion:":<23} {CONFUSION}'
    y = build_text(draw, font, texts, text, x, y)

    text = f'{"coreid:":<23} {rec["coreid"]}'
    y = build_text(draw, font, texts, text, x, y)

    for field, label in FIELDS:
        text = f'{(label+":"):<23} {rec[field]}'
        y = build_text(draw, font, texts, text, x, y)

    flag = ''
    if rec['fruiting']:
        flag += '1'
    elif rec['not_fruiting']:
        flag += '0'

    text = f'{"fruiting NLP:":<23} {flag}'
    y = build_text(draw, font, texts, text, x, y)

    text = f'{"fruiting model:":<23} {round(rec["pred"])} ({rec["pred"]:0.4})'
    y = build_text(draw, font, texts, text, x, y)

    max_x = max(t.bbox[2] for t in texts)
    draw.rectangle((0, 0, max_x + 32, y + 32), fill='#eeeeee')

    for t in texts:
        draw.text((t.x, t.y), t.text, color, font=font)

    path = DIR / f'{rec["coreid"]}.jpg'
    image.save(path, 'JPEG')

    # display(image)


# fruiting(19)
# interact(fruiting, idx=(1, len(dataset)))

In [57]:
for i, _ in enumerate(DATASET[:20], 1):
    fruiting(i)